In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import os
import sys
import jax.tree_util as jtu
import subprocess

In [2]:
sys.path.append(os.path.dirname(os.getcwd()))
from ml_collections import ConfigDict
from rdkit import Chem
from train.scheduler import GaussianDiffusion
from train.inference import smi2graph_features
from train.rewards import smi2pdbqt, dsdp_reward

In [3]:
lead_smi = 'O=C1C=C(OC2=C1C=CC=C2C1=CC=CC=C1)N1CCOCC1'
lead_smi = Chem.MolToSmiles(Chem.MolFromSmiles(lead_smi), canonical=True)
print("Canonical SMILES: ", lead_smi)

Canonical SMILES:  O=c1cc(N2CCOCC2)oc2c(-c3ccccc3)cccc12


In [4]:
def dsdp_reward(smi: str, cached_file_path: str, dsdp_script_path: str):
    smi_save_dir = os.path.join(cached_file_path, f'temp-ligand.pdbqt')
    smi2pdbqt(smi, smi_save_dir)
    out_dir = os.path.join(cached_file_path, f'temp-dock.pdbqt')
    log_dir = os.path.join(cached_file_path, f'temp-log.log')
    cmd = ['bash', dsdp_script_path, smi_save_dir, out_dir, log_dir]
    cmd = ' '.join(cmd)
    subprocess.run(cmd, check=True, shell=True)
    with open(log_dir, 'r') as f:
        lines = f.readlines()
        ss = [float(s.split()[-1]) for s in lines]
    return ss[0]

In [5]:
os.makedirs('./init-molecule', exist_ok=True)
root_wd = os.path.dirname(os.getcwd())
graphs = smi2graph_features(lead_smi)
score = [
    dsdp_reward(lead_smi, f'{root_wd}/tests/init-molecule', f'{root_wd}/cases/case_parp1-pik3ca/dsdp_parp1.sh'),
    dsdp_reward(lead_smi, f'{root_wd}/tests/init-molecule', f'{root_wd}/cases/case_parp1-pik3ca/dsdp_pik3ca.sh'),
]
score = np.asarray(score, np.float32) * (-1)
print(score)

/home/dvnguye2/MolSculptor/tests/init-molecule/temp-ligand.pdbqt
Total time 1.420622 s
/home/dvnguye2/MolSculptor/tests/init-molecule/temp-ligand.pdbqt
Total time 1.445829 s
[9.794899 9.422937]


In [6]:
molecule = {
        'scores': np.asarray(score, np.float32),
        'smiles': np.asarray(lead_smi, object),
        'graphs': graphs,
    }

In [7]:
with open(f'/home/dvnguye2/MolSculptor/cases/case_parp1-pik3ca/init_search_molecule.pkl', 'wb') as f:
    pkl.dump(molecule, f)